In [1]:
import torch
import numpy as np
import random
import os

seed = 2321321

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [2]:
# load entities and relations
import json

path = '../data/'

with open(path+'entities.json', 'r') as fin:
    entities = json.load(fin)
print(len(entities), 'entities')

with open(path+'properties.json', 'r') as fin:
    properties = json.load(fin)
print(len(properties), 'properties')

28497 entities
8913 properties


In [3]:
# load KG
from hdt import HDTDocument, TripleComponentRole

hdt_path = "/ivi/ilps/personal/svakule/"
hdt_file = 'wikidata2018_09_11.hdt'
# hdt_file = 'wikidata20200309.hdt'

PREFIX_E = 'http://www.wikidata.org/entity/'
PREFIX_P = 'http://www.wikidata.org/prop/direct/P'

kg = HDTDocument(hdt_path+hdt_file)

# Display some metadata about the HDT document itself
print("nb triples: %i" % kg.total_triples)
print("nb subjects: %i" % kg.nb_subjects)
print("nb predicates: %i" % kg.nb_predicates)

# load labels
import json

path = '../data/'
max_triples = 50000
offset = 0
namespace = 'predef-wikidata2018-09-all' # 'predef-wikidata2020-03-all'  # 

import sympy

import scipy.sparse as sp


def generate_adj_sp(adjacencies, n_entities, include_inverse):
    '''
    Build adjacency matrix
    '''
    adj_shape = (n_entities, n_entities)
    
    # colect all predicate matrices separately into a list
    sp_adjacencies = []
    for edges in adjacencies:
        # split subject (row) and object (col) node URIs
        n_edges = len(edges)
        row, col = np.transpose(edges)
        
        # duplicate edges in the opposite direction
        if include_inverse:
            _row = np.hstack([row, col])
            col = np.hstack([col, row])
            row = _row
            n_edges *= 2
        
        # create adjacency matrix for this predicate
        data = np.ones(n_edges)
        adj = sp.csr_matrix((data, (row, col)), shape=adj_shape)
        sp_adjacencies.append(adj)
    
    return np.asarray(sp_adjacencies)

nb triples: 2935160017
nb subjects: 760717318
nb predicates: 23387


In [4]:
# formulate hypotheses: 1k cutoff
def retrieve_answer(audio_file_path, hyp_cutoff=100, rerank_cutoff=0):
    # formulate hypotheses for entities and properties based on BP decoding
    e_hypotheses, p_hypotheses = w2v_recognizer.score_labels(audio_file_path)
    e_hypotheses = e_hypotheses.most_common(hyp_cutoff)
    p_hypotheses = p_hypotheses.most_common(hyp_cutoff)
    
    # get all entity labels
#     print(e_hypotheses[:2])
    e_hypotheses = [l.upper() for l, c in e_hypotheses]
    # rescoring
    e_rank = w2v_recognizer.rescore(e_hypotheses)
    
    matched_entity_ids = []
    matched_entity_scores = []
    e_ids = []
    for j in range(len(e_rank)):
        label = e_hypotheses[e_rank[j]]
        score = 1 / (j + 1)
        if score > rerank_cutoff:
            # look up entites in the KG by label
#             print(label, score)
            for e_id in w2v_recognizer.entities_labels2ids[label.lower()]:
                if e_id not in e_ids:
                    e_ids.append(e_id)
                    entity = PREFIX_E + e_id
                    kg_id = kg.string_to_global_id(entity, TripleComponentRole.OBJECT)
                    matched_entity_ids.append(kg_id)
                    matched_entity_scores.append(score)
            
#     print(len(p_hypotheses), 'properties')
    # look up predicate ids in the KG
    matched_relation_ids = []
    matched_relation_scores = []
    p_ids = []
    for p_label, c in p_hypotheses:
        # look up by label
#         print(p_label, c)
        for p_id in w2v_recognizer.relations_labels2ids[p_label]:
            p_id = p_id[1:]
            if p_id not in p_ids:
                p_ids.append(p_id)
                predicate = PREFIX_P+p_id
                matched_relation_ids.append(kg.string_to_global_id(predicate, TripleComponentRole.PREDICATE))
                matched_relation_scores.append(c)
                
    kg.configure_hops(1, matched_relation_ids, namespace, True, False)
    subgraph = kg.compute_hops(matched_entity_ids, max_triples, offset)
    entity_ids, predicate_ids, adjacencies = subgraph
    
    _max = 1 #torch.max(-loss, dim=-1).values.item()
    n_entities = len(entity_ids)
    n_relations = len(predicate_ids)
    predicted = []
    if n_entities != 0 and n_relations != 0:
        # activations
        x = np.zeros(n_entities)
        for i, e in enumerate(matched_entity_ids):
            if e in entity_ids:
                idx = entity_ids.index(e)
                x[idx] = matched_entity_scores[i]
                
        p = np.zeros(n_relations)
        for i, e in enumerate(matched_relation_ids):
            if e in predicate_ids:
                idx = predicate_ids.index(e)
                p[idx] = matched_relation_scores[i]

        # load adjacencies
        A = generate_adj_sp(adjacencies, n_entities, include_inverse=True)
        y = np.zeros(n_entities)
        for i, _A in enumerate(A):
            _A = p[i] * _A
            # MP
            _y = x @ _A
            _y[_y>_max] = _max
#             y += _y
#             print(_y)
            y = np.vstack([y, _y])
        y = np.amax(y, 0)
#         print(y)
        answer_scores = []
        for e in (-y).argsort():  # sorts ascending
            # exclude activated entities
            if x[e] == 0:
                answer = kg.global_id_to_string(entity_ids[e], TripleComponentRole.OBJECT)
    #             print(answer)
                answer = answer.split('/')[-1]
                if answer in entities:
                    predicted.append(entities[answer])
                    answer_scores.append(y[e])
        return predicted, answer_scores, e_ids, p_ids
        # DONE

In [5]:
# from ctc_search import Recognizer
import torch.nn as nn


from ctc_search_loss import Recognizer

TOPN = 100  # number of candidate labels to consider as hypotheses space 500

# load pre-trained ASR model
w2v_recognizer = Recognizer(topn=TOPN)

w2v_recognizer.model = nn.DataParallel(w2v_recognizer.model)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
import numpy as np

from utils import preprocess

wav_path = "../data/gtts/sqs_valid_wikidata2018_09_11/"

e_EM, r_EM, a_EM = 0, 0, 0
e_rs, r_rs, a_rs = 0, 0, 0
e_rrs, r_rrs, a_rrs = [], [], []

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
        
w2v_recognizer.model.to('cuda')
w2v_recognizer.model.eval()

with open('../data/sqs_valid_wikidata2018_09_11.txt') as fin:
    lines = fin.readlines()
    print(len(lines), 'samples')

answer_h1 = 0
answer_h3 = 0
answer_h5 = 0
answer_h10 = 0
entity_recall = 0
property_recall = 0

n = 0

for i, l in enumerate(lines[:]):
    #         print(i)
    s, p, o, q = l.strip('\n').split('\t')
    p = p[1:]
    
    if s in entities and p in properties:
        n += 1
        # load ground truth
        s_label = preprocess(entities[s])
        p_label = preprocess(properties[p])
        a_label = entities[o]

        print(q)

        # predict distribution
        file = wav_path + str(i+1) + '.wav'
        answers, answer_scores, e_ids, p_ids = retrieve_answer(file)
        
        print('Entity', s in e_ids)
        print('Property', p in p_ids)
        
        if answers:
            answer_h1 += int(a_label == answers[0])
            answer_h3 += int(a_label in answers[:3])
            answer_h5 += int(a_label in answers[:5])
            answer_h10 += int(a_label in answers[:10])
            print(answers[0], a_label)
            print('Hit@1', a_label == answers[0])
            print('Hit@5', a_label in answers[:10], '\n')
        entity_recall += int(s in e_ids)
        property_recall += int(p in p_ids)
    
        
print(n, 'samples')
print('Hits@1:', answer_h1/n)
print('Hits@3:', answer_h3/n)
print('Hits@5:', answer_h5/n)
print('Hits@10:', answer_h10/n)
print('Entity recall:', entity_recall/n)
print('Property recall:', property_recall/n)

Let's use 4 GPUs!
1026 samples
where was sasha vujačić born
where was sasha vijasic born
Entity True
Property True
Maribor Maribor
Hit@1 True
Hit@5 True 

What was Seymour Parker Gilbert's profession?
what was semo parkegilbet's profession
Entity True
Property True
Woody Strode lawyer
Hit@1 False
Hit@5 True 

in what french city did antoine de févin die 
in what french city did untwinde feaven die
Entity False
Property False
male Blois
Hit@1 False
Hit@5 False 

who is the chid of fritz leiber?
who is the chit of frit sleeper
Entity False
Property True
Laurie R. King Justin Leiber
Hit@1 False
Hit@5 False 

What was bill hosket, jr.'s position 
what was bill hoskit junior's position
Entity True
Property True
defender center
Hit@1 False
Hit@5 False 

Which genre is teri meri kahaani under
which jonra is terry mery cahaneander
Entity False
Property False
male romance film
Hit@1 False
Hit@5 False 

what's the name of a founder of ndtv
what's the name of a founder of end t t
Entity False
Pro

Entity True
Property False
screenwriter fantasy
Hit@1 False
Hit@5 False 

What is matt gentry's gender?
what is mat gentry's gender
Entity True
Property True
male male
Hit@1 True
Hit@5 True 

What is Michiyo Okusu's gender?
what is mitcheoacuses gender
Entity False
Property True
male female
Hit@1 False
Hit@5 True 

What language is spoken in yesterday
what language is spoken in yesterday
Entity True
Property True
English Zulu
Hit@1 False
Hit@5 True 

What is Nikola Rachelle's gender?
what is nicholarishell's gender
Entity False
Property True
male female
Hit@1 False
Hit@5 True 

What position does iván malón play?
what position does ivan malen play
Entity True
Property True
association football defender
Hit@1 False
Hit@5 True 

who is the record label of the artist the summer set?
who is the record label of the artist the summerset
Entity True
Property True
United States of America Fearless Records
Hit@1 False
Hit@5 False 

where did ludwig stickelberger die
where did lok vixed taccober

Entity False
Property True
Frank O'Connor Lee "Scratch" Perry
Hit@1 False
Hit@5 False 

What is a hub of the airline baboo (airline)
what is a hub of the airline purbo airline
Entity False
Property True
Juan Santamaría International Airport Geneva International Airport
Hit@1 False
Hit@5 False 

what kind of music is aziatix known for
what kind of music is as the attics known for
Entity False
Property True
musical film rhythm and blues
Hit@1 False
Hit@5 False 

where does bianca lattuada come from
where does be ancho latuade come from
Entity True
Property False
People's Republic of China Italy
Hit@1 False
Hit@5 False 

what type of film is robin hood 
what type of film is robin hurt
Entity True
Property True
country music silent film
Hit@1 False
Hit@5 False 

which asteroid group contains 13070 seanconnery
which asteroid group contains thirteen thousand and seventy shorn conery
Entity False
Property True
male asteroid belt
Hit@1 False
Hit@5 False 

what is valerie vaz's country of natio

Entity True
Property True
drama drama
Hit@1 True
Hit@5 True 

What religion does maria shriver practice
what religion does maria shriver practice
Entity True
Property True
Boston Catholicism
Hit@1 False
Hit@5 True 

Where was frederick bayer born?
where was frederick byr born
Entity True
Property True
male Asbury Park
Hit@1 False
Hit@5 True 

Where did melvil dewey die?
where did melville dewy dye
Entity True
Property False
male Lake Placid
Hit@1 False
Hit@5 False 

What is manuel lisa's nationality
what is manwell lease's nationality
Entity False
Property True
United States of America United States of America
Hit@1 True
Hit@5 True 

whats the name of ken saro-wiwa's child
what's the name of cansarow wiwer's child
Entity False
Property True
screenwriter Ken Wiwa
Hit@1 False
Hit@5 False 

who is the artist of the album the chemical wedding?
who is the artist of the album the chemical wedding
Entity True
Property True
Bruce Dickinson Bruce Dickinson
Hit@1 True
Hit@5 True 

Who directed t

Entity True
Property True
comedy film soul music
Hit@1 False
Hit@5 False 

what is the sex of ed peck
what is the saxovad pack
Entity False
Property False
Josip Runjanin male
Hit@1 False
Hit@5 False 

Where in texas was kevin ollie born
wherein taxis was cavern orly born
Entity False
Property True
baseball player Dallas
Hit@1 False
Hit@5 False 

what positions has jim mclaughlin played 
what positions has jim mec clochlan played
Entity False
Property True
Everything Will Never Be OK forward
Hit@1 False
Hit@5 False 

what is the place of death of geneviève winding
what is the place of death of genovive winding
Entity True
Property True
male Saint-Cloud
Hit@1 False
Hit@5 True 

What is john kean's gender?
what is john keen's gender
Entity True
Property True
male male
Hit@1 True
Hit@5 True 

what is marthinus van schalkwyk's job
what is marfines van scok vix job
Entity False
Property True
male politician
Hit@1 False
Hit@5 True 

where was marcelo garcia born
where was massellar gassier bo

...

Entity False
Property True
botanist James Prescott Joule
Hit@1 False
Hit@5 False 

What language is in ever ever after
what languages in ever ever after
Entity False
Property True
United States of America English
Hit@1 False
Hit@5 True 

Where was terry lloyd when he died
where was terry loyd when he died
Entity True
Property True
Todd Field Basra
Hit@1 False
Hit@5 False 

What is michelle rempel's nationality? 
what is michel rample's nationality
Entity False
Property True
male Canada
Hit@1 False
Hit@5 False 

Who wrote the cry of the owl
he wrote the cry of the owl
Entity True
Property True
Claude Chabrol Claude Chabrol
Hit@1 True
Hit@5 True 

what is bruce mcdonald (film director)'s nationality
what is bruce mac donald film director s nationality
Entity True
Property True
United States of America Canada
Hit@1 False
Hit@5 True 

is hanns ludin a male or female
is hands liw don a male or female
Entity False
Property True
Shōko Wakabayashi male
Hit@1 False
Hit@5 False 

WHich nation fi

Entity True
Property True
music video game thrash metal
Hit@1 False
Hit@5 False 

Who contributed to music on the film walking and talking?
who contributed to music on the film walking and talking
Entity False
Property True
United States of America Billy Bragg
Hit@1 False
Hit@5 False 

where was péter kornai born
where was patter corni born
Entity False
Property True
United States of America Budapest
Hit@1 False
Hit@5 False 

What country is the tv show dracula from?
what country is the t v show dracula from
Entity True
Property True
United States of America United Kingdom
Hit@1 False
Hit@5 True 

who is the child of thomas cromwell
he is the child of thomas cromwell
Entity True
Property True
male Gregory Cromwell, 1st Baron Cromwell
Hit@1 False
Hit@5 True 

who produced the film short term 12
he produced the film short term twelve
Entity True
Property True
The Matrix Reloaded Joshua Astrachan
Hit@1 False
Hit@5 False 

what is bruno cerella's country of origin?
what is bruno serella's 

Entity True
Property True
independent music J-pop
Hit@1 False
Hit@5 False 

what country is cusco region located 
what country is cuscoe region located
Entity False
Property True
United States of America Peru
Hit@1 False
Hit@5 False 

what instrument does mark matejka play 
what instrument is mark mateoico play
Entity False
Property True
United States of America guitar
Hit@1 False
Hit@5 False 

who wrote the story for la perla?
who wrote the story por lipurla
Entity False
Property False
Loudon Wainwright III John Steinbeck
Hit@1 False
Hit@5 False 

who wrote the song black diamond bay
he wrote the song black diamond bay
Entity True
Property False
Mark Isham Bob Dylan
Hit@1 False
Hit@5 False 

in what country was treasure of the amazon filmed
in what country was treasure of the amazin filmed
Entity True
Property True
United States of America United States of America
Hit@1 True
Hit@5 True 

what conflict was francis william farrell involved in
what conflict was francis william farrell in

what is a gameplaymod that can be played on tree on cue
Entity False
Property True
American football single-player video game
Hit@1 False
Hit@5 False 

what is the gender of  zak thompson
what is the gender of zac thompson
Entity True
Property True
male male
Hit@1 True
Hit@5 True 

what country is langston hughes from
what country is langston heres from
Entity True
Property False
United States of America United States of America
Hit@1 True
Hit@5 True 

what sport does the team cis play
what sport does the team siss play
Entity False
Property True
Souls at Sea rugby union
Hit@1 False
Hit@5 False 

What country was up! released?
what country was up released
Entity True
Property True
United States of America United States of America
Hit@1 True
Hit@5 True 

What is iota crucis?
what is io ta crusis
Entity True
Property False
Canada star
Hit@1 False
Hit@5 False 

What kind of cancer killed bonita granville
what kind of cancer killed benita granvelle
Entity True
Property False
Santa Monica l

Entity True
Property True
Princeton University Philadelphia
Hit@1 False
Hit@5 True 

is dani osvaldo male or female
is danniospalta male or female
Entity True
Property True
association football male
Hit@1 False
Hit@5 False 

What is the location of clarence streit's death
what is the location of clarence stripes death
Entity True
Property True
actor Washington, D.C.
Hit@1 False
Hit@5 False 

what type of computer videogame genre is the game  red dead redemption: undead nightmare
what type of computer vidio game jonra is the game red dead redemption undead nightmare
Entity True
Property False
Imagine Entertainment action-adventure game
Hit@1 False
Hit@5 False 

where is abby mann's nationality
where is appy man's nationality
Entity False
Property True
United Kingdom United States of America
Hit@1 False
Hit@5 True 

what is the sex of seumas o'kelly?
what is the sex of seamazoceli
Entity False
Property True
Sean Connery male
Hit@1 False
Hit@5 True 

which position did herby fortunat play